# 1. Setup

## 1.1 Module import

Import necessary modules

In [ ]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from keras.applications.mobilenet_v3 import preprocess_input

## 1.2 Dataset import

Define parameters for image import:

In [ ]:
batch_size = 32
img_width = 80
img_height = 80

data_dir = '/mnt/d/Dev/DHBW/DHBW_Studienarbeit_NN_Development/images/per_label'

OPTIONAL: Force tensorflow to use the CPU. Can be useful if you want to compare the performance of GPU and CPU

In [ ]:
tf.config.set_visible_devices([], 'GPU')

Import dataset for training

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode = 'categorical',
    validation_split = 0.1,
    subset = 'training',
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Import dataset for validation

In [ ]:
val_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode = 'categorical',
    validation_split = 0.1,
    subset = 'validation',
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

# 2. Transfer learning

## 2.1 Construct the new model

Load the base model

In [ ]:
IMG_SIZE = (img_height, img_width)
IMG_SHAPE = IMG_SIZE + (3,)

base_model = tf.keras.applications.MobileNetV3Small(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

Add the custom classification layer to it

In [ ]:
# new layers
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(3, activation = 'softmax')

# construct the final model
inputs = keras.Input(shape = (img_height, img_width, 3))
x = preprocess_input(inputs)
x = base_model(x, training = False)
x = global_average_layer(x)
outputs = prediction_layer(x)
x = tf.keras.layers.Dropout(0.05)(x)
model = keras.Model(inputs, outputs)

model.summary()

## 2.2 Start the initial learning process

Define Parameters for the learning process

In [ ]:
base_learning_rate = 0.02
initial_epochs = 25

Compile the before built model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

Train the model!

In [ ]:
history = model.fit(train_ds, epochs=initial_epochs, validation_data=val_ds)

# 3. Fine tuning the base model

Freeze all layers except for the uppermost layers of the base model. Layzers will be frozen from ***fine_tune_from*** to ***len(base_model.layers)***.

In [ ]:
base_model.trainable = True
fine_tune_from = len(base_model.layers) - 20

for layer in base_model.layers[:fine_tune_from]:
    layer.trainable = False

# don't train the dense layer
model.layers[len(model.layers) - 1].trainable = False

model.summary()

Compile the model again but decrease the learning rate in order to avoid overfitting.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/1000), loss='categorical_crossentropy', metrics=['accuracy'])

Train the uppermost layers of the base model and proceed where you left off in the initial training by using the last epoch of the first training as the initial epoch for fine tuning.

In [ ]:
history = model.fit(train_ds, epochs=initial_epochs + 10,  initial_epoch=history.epoch[-1], validation_data=val_ds)

# 4. Save the model

Info: Loading can be done via: ***model = tf.keras.models.load_model(PATH_TO_MODEL_FILE)***


In [ ]:
# for now until it is implemented properly: manually make sure that the path exists (in this case '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/')
save_path = '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/my_model.keras'

model.save(save_path)